In [ ]:
import numpy as np
import itertools
from tabulate import tabulate

# Permutations test
Nous allons créer une petite fonction permettant d'évaluer et de comprendre nos métriques sur des données générées informatiquement. Il s'agit de regarder les valeurs des métriques sur toutes les permutations de $[\![1 , n ]\!]$.


In [ ]:
def all_perms(elements):
    '''
    Entrée : elements -- un tableau de nombre
    Sortie: un générateur contenant toutes les permutations (listes) du tableau en entrée
    '''
    if len(elements) <=1:
        yield elements
    else:
        for perm in all_perms(elements[1:]):
            for i in range(len(elements)):
                yield perm[:i] + elements[0:1] + perm[i:]


In [ ]:
def print_array_perm(n, metrics, sorted=False):
  '''
  Affiche et renvoie une liste de toutes les permutations de [1,n] et les évalue selon les metriques
  Entrée :
    n -- Entier
    metrics -- un tableau de string avec les noms des metrics
    sorted -- indique si l'on veut sort selon le première métrique du tableau metrics
  Sortie : 
    array_metrics -- Un tableau de taille (2**n, m+1) où m est la taille du tableau metrics 
  '''
  array_1n = list(range(1,n+1))
  array_perm = list(all_perms(array_1n))
  m = len(metrics)
  array_metrics = [[metric(perm) for metric in metrics] for perm in array_perm]
  if sorted:
    array_metrics = array_metrics[np.argsort(array_perm[:,0])]
  titles = ['Ordre d\'arrivée'] + [metric.__name__ for metric in metrics]
  format_row = "{:>16}"*(m+1)
  print(format_row.format(*titles))
  format_row = "{:>12}"*(m+1)
  for ordre, metrics_values in zip(array_perm, array_metrics):
    print(format_row.format(str(ordre), *metrics_values))
  #return(array_perm, array_metrics)
 

On peut utiliser tabular, qui fonctionne très bien et donne facilement accès à davantage de fonctions d'affichage

In [ ]:
def print_array_perm2(n, metrics, sorted=False):
  '''
  Affiche et renvoie une liste de toutes les permutations de [1,n] et les évalue selon les metriques
  Entrée :
    n -- Entier
    metrics -- un tableau de string avec les noms des metrics
    sorted -- indique si l'on veut sort selon le première métrique du tableau metrics (du plus grand au plus petit)
  Sortie : 
    array_metrics -- Un tableau de taille (2**n, m+1) où m est la taille du tableau metrics 
  '''

  array_1n = list(range(1,n+1))
  array_perm = list(all_perms(array_1n))
  
  m = len(metrics)
  #attention, array_metrics est un tableau de string
  array_metrics = np.array([[str(perm)]+[metric(perm) for metric in metrics] for perm in array_perm])
  
  if sorted:
    array_metrics1 = array_metrics[np.argsort(list(map(float,array_metrics[:,1])))[::-1]]

  titles = ['Ordre d\'arrivée'] + [metric.__name__ for metric in metrics]
  print(tabulate(array_metrics1, headers=titles))
  return()
  #return(array_perm, array_metrics)

# Metriques Learning to Rank

---




On implémente différentes métriques :
* Normalized Discounted Cumulative
Gain (NDCG) [(Jarvelin and Kekalainen, 2002)](https://dl.acm.org/doi/10.1145/582415.582418). Ici, les queries sont des courses, les documents sont les chevaux de la course, et les relevance sont n - la position (ainsi, le premier est le plus relevant)
* winner (Le classement a-t-il prédit le gagnant ?)

In [ ]:
def DCG(results, n=-1):
  '''
  Implémente le calcul du Discounted Cumulative Gain
  Entrée :
    results -- liste de résultats où les index correspondent au classement prédit et les valeurs sont les résultats réels de la course
    n -- la limite de la somme, par défaut tous les chevaux sont pris en compte
  '''
  if n == -1:
    n = len(results)
  top = results[:n]
  return(np.sum(np.array([len(top)-top[i]/(np.log(i+2)) for i in range(len(top))])))

In [ ]:
def eDCG(results, n=-1):
  '''
  Implémente le calcul du Discounted Cumulative Gain en pondérant le classement réel de manière exponentielle (les premiers classements ont plus d'importance)
  '''
  if n == -1:
    n = len(results)
  top = results[:n]
  return(np.sum(np.array([(2**(len(top)-top[i])-1)/(np.log(i+2)) for i in range(len(top))])))

In [ ]:
def winner(results):
  return(int(results[0]==1))

In [ ]:
def NDCG(results, n=-1):
  '''
  Compute le Normalized Discouted Cumulative Gain. Le gain idéal est celui du classement parfait.
  '''
  if n == -1:
    n = len(results)
  top = results[:n]
  return(DCG(results, n)/DCG(list(range(1,n)), n))


In [ ]:
eDCG([8,7,6,5,4,3,2,1])

119.82292109108094

In [ ]:
print_array_perm2(4, [eDCG, NDCG, winner], sorted=True)

Ordre d'arrivée              eDCG     NDCG    winner
------------------------  -------  -------  --------
[1, 2, 3, 4, 5, 6, 7, 8]  278.179  1.94504         1
[1, 2, 3, 4, 5, 6, 8, 7]  278.153  1.94388         1
[1, 2, 3, 4, 5, 7, 6, 8]  278.113  1.94356         1
[1, 2, 3, 4, 5, 8, 6, 7]  278.054  1.94091         1
[1, 2, 3, 4, 5, 7, 8, 6]  278.035  1.94124         1
[1, 2, 3, 4, 5, 8, 7, 6]  278.002  1.93975         1
[1, 2, 3, 4, 6, 5, 7, 8]  278.002  1.94305         1
[1, 2, 3, 4, 6, 5, 8, 7]  277.976  1.94189         1
[1, 2, 3, 4, 7, 5, 6, 8]  277.847  1.93958         1
[1, 2, 3, 4, 6, 7, 5, 8]  277.804  1.94008         1
[1, 2, 3, 4, 7, 5, 8, 6]  277.77   1.93726         1
[1, 2, 3, 4, 6, 8, 5, 7]  277.745  1.93744         1
[1, 2, 3, 4, 8, 5, 6, 7]  277.744  1.93494         1
[1, 2, 3, 4, 7, 6, 5, 8]  277.715  1.93809         1
[1, 2, 3, 4, 8, 5, 7, 6]  277.693  1.93378         1
[1, 2, 3, 5, 4, 6, 7, 8]  277.673  1.9422          1
[1, 2, 3, 5, 4, 6, 8, 7]  277.647  1.94104    

Pour évaluer les métriques, il faudra faire les moyennes des DCG, prendre en compte la précision, prendre en compte la précision comparée au public, prendre en compte les chevaux qui sont tombés.

Analyse de NCDG :
Moins pénalisant de s
eDCG 

Idées :  
prédire le temps d'arrivée (en normalisant)  
elo : prendre en compte les temps d'arrivée